In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델과 토크나이저 불러오기
model_name = "yanolja/EEVE-Korean-10.8B-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모델 로드 (device_map="auto"로 자동 배분)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    load_in_8bit=True,  # 8-bit quantization 사용
    device_map="auto"   # 자동으로 GPU/CPU에 배분
)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /opt/conda/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:

# 프롬프트
prompt = """다음 대화를 한국어로 20퍼센트 정도로 요약해 주세요: Person1#: 안녕하세요, 오늘 하루 어떠셨어요? 
Person2#: 요즘 숨쉬기가 좀 힘들어요.
Person1#: 최근에 감기 같은 것에 걸리신 적이 있나요?
Person2#: 아니요, 감기는 아니에요. 그냥 숨을 쉴 때마다 가슴이 무겁게 느껴져요.
Person1#: 알고 있는 알레르기가 있나요?
Person2#: 아니요, 알고 있는 알레르기는 없어요.
Person1#: 이런 증상이 항상 나타나나요, 아니면 활동할 때 주로 나타나나요?
Person2#: 운동을 할 때 많이 나타나요.
Person1#: 저는 당신을 폐 전문의에게 보내서 천식에 대한 검사를 받게 할 거예요.
Person2#: 도와주셔서 감사합니다, 의사 선생님."""


# 입력 프롬프트를 토큰화하고, max_length 설정
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to("cuda")

# 모델을 사용해 응답 생성 (pad_token_id를 eos_token_id로 설정)
output_tokens = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id
)

# 출력 결과 디코딩
output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

# 결과 출력
print(output)

다음 대화를 한국어로 20퍼센트 정도로 요약해 주세요: Person1#: 안녕하세요, 오늘 하루 어떠셨어요? 
Person2#: 요즘 숨쉬기가 좀 힘들어요.
Person1#: 최근에 감기 같은 것에 걸리신 적이 있나요?
Person2#: 아니요, 감기는 아니에요. 그냥 숨을 쉴 때마다 가슴이 무겁게 느껴져요.
Person1#: 알고 있는 알레르기가 있나요?
Person2#: 아니요, 알고 있는 알레르기는 없어요.
Person1#: 이런 증상이 항상 나타나나요, 아니면 활동할 때 주로 나타나나요?
Person2#: 운동을 할 때 많이 나타나요.
Person1#: 저는 당신을 폐 전문의에게 보내서 천식에 대한 검사를 받게 할 거예요.
Person2#: 도와주셔서 감사합니다, 의사 선생님.

Person1: Hello, how was your day?
Person2: I'm having trouble breathing lately.
Person1: Have you had any recent illnesses like a cold?
Person2: No, it's not a cold. It just feels like my chest is heavy when I breathe.
Person1: Do you have any known allergies?
Person2: No, I don't have any known allergies.
Person1: Does this symptom
